In [6]:
from transformers import AutoTokenizer
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import json
import random

def read_jsonl_dataset(file_path):
    """Чтение JSONL датасета """
    sentences = []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            item = json.loads(line.strip())
            tokens = item['tokens']
            ner_tags = item['ner_tags']
            
            # Создаем предложение в формате (word, tag)
            sentence = [(token, tag) for token, tag in zip(tokens, ner_tags)]
            sentences.append(sentence)
    
    print(f"Загружено {len(sentences)} предложений из JSONL файла")
    return sentences

def create_label_mapping(sentences):
    """Создание mapping между метками и ID"""
    all_labels = set()
    for sentence in sentences:
        for word, label in sentence:
            all_labels.add(label)
    
    # Сортируем метки, начиная с 'O'
    label_list = ['O'] + sorted([l for l in all_labels if l != 'O'])
    label2id = {label: i for i, label in enumerate(label_list)}
    id2label = {i: label for label, i in label2id.items()}
    
    print(f"Найдено {len(label_list)} меток: {label_list}")
    return label2id, id2label, label_list

def align_labels_with_tokens(word_ids, labels):
    """Выравнивает метки с токенами"""
    aligned_labels = []
    previous_word_idx = None
    
    for word_idx in word_ids:
        if word_idx is None:
            aligned_labels.append(-100)
        elif word_idx != previous_word_idx:
            aligned_labels.append(labels[word_idx])
        else:
            aligned_labels.append(-100)
        previous_word_idx = word_idx
    
    return aligned_labels

def create_sliding_windows_complete(words, labels, tokenizer, window_size=510, stride=256):
    """Создает перекрывающиеся окна, сохраняя все части текста"""
    
    # Токенизируем весь текст 
    encoding = tokenizer(
        words,
        is_split_into_words=True,
        truncation=False,
        padding=False,
        return_offsets_mapping=True
    )
    
    full_tokens = encoding['input_ids']
    word_ids = encoding.word_ids()
    
    # Если текст короткий - возвращаем как есть
    if len(full_tokens) <= window_size:
        aligned_labels = align_labels_with_tokens(word_ids, labels)
        return [(full_tokens, aligned_labels)]
    
    windows = []
    
    # Создаем перекрывающиеся окна до самого конца
    start_idx = 0
    while start_idx < len(full_tokens):
        end_idx = min(start_idx + window_size, len(full_tokens))
        
        # Cохраняем все окна, даже короткие в конце
        window_tokens = full_tokens[start_idx:end_idx]
        window_word_ids = word_ids[start_idx:end_idx]
        
        # Выравниваем метки для этого окна
        window_labels = []
        previous_word_idx = None
        
        for word_idx in window_word_ids:
            if word_idx is None:
                window_labels.append(-100)
            elif word_idx != previous_word_idx:
                window_labels.append(labels[word_idx])
            else:
                window_labels.append(-100)
            previous_word_idx = word_idx
        
        windows.append((window_tokens, window_labels))
        
        if end_idx == len(full_tokens):
            break
            
        # Сдвигаем окно
        start_idx += stride
        
        # Гарантируем, что последнее окно захватывает конец текста
        if start_idx + window_size >= len(full_tokens) and start_idx < len(full_tokens):
            start_idx = max(0, len(full_tokens) - window_size)
    
    #print(f"Создано {len(windows)} окон для текста из {len(full_tokens)} токенов")
    return windows

def prepare_dataset_with_sliding_windows(jsonl_file_path, tokenizer, window_size=510, stride=256, val_fraction=0.2):
    """Делит датасет на окна, а затем на трейн и валидацию, гарантируя, 
        что все окна одного документа будут только в одной из двух частей"""
    
    print("Подготовка данных с перекрывающимися окнами...")
    
    # 1. Чтение данных
    sentences = read_jsonl_dataset(jsonl_file_path)
    print(f"Загружено {len(sentences)} исходных документов")
    
    # 2. Создание mapping меток
    label2id, id2label, label_list = create_label_mapping(sentences)
    
    # 3. Нарезаем на окна, сохраняем индекс документа
    all_windows = []
    all_labels = []
    doc_indices = []  # Для каждого окна храним индекс документа
    
    stats = {
        'total_docs': len(sentences),
        'windows_per_doc': []
    }
    
    for doc_idx, sentence in enumerate(sentences):
        words = [word for word, label in sentence]
        label_ids = [label2id[label] for word, label in sentence]
        
        # Создаем окна для этого документа
        windows = create_sliding_windows_complete(words, label_ids, tokenizer, window_size, stride)
        
        # Сохраняем каждое окно с индексом документа
        for window_tokens, window_labels in windows:
            all_windows.append(window_tokens)
            all_labels.append(window_labels)
            doc_indices.append(doc_idx)
        
        stats['windows_per_doc'].append(len(windows))
    
    total_windows = len(all_windows)
    print(f"\nСтатистика по окнам:")
    print(f"Всего окон: {total_windows}")
    print(f"Среднее число окон на документ: {sum(stats['windows_per_doc'])/len(sentences):.1f}")
    print(f"Мин/макс окон на документ: {min(stats['windows_per_doc'])} / {max(stats['windows_per_doc'])}")
    
    # 4. Набираем валидацию целыми документами
    target_val_windows = int(total_windows * val_fraction)
    print(f"\nЦелевой размер валидации: {target_val_windows} окон ({val_fraction*100:.0f}%)")
    
    # Получаем уникальные индексы документов и перемешиваем
    unique_docs = list(set(doc_indices))
    random.seed(42)
    random.shuffle(unique_docs)
    
    # Набираем документы в валидацию, пока не достигнем цели
    val_doc_indices = set()
    val_windows_count = 0
    docs_used = []
    
    for doc_idx in unique_docs:
        # Сколько окон дает этот документ
        doc_window_count = stats['windows_per_doc'][doc_idx]

        # Проверяем, что с добавлением документа, 
        # число окон не будет превышать требуемое более, чем на 10%
        if val_windows_count + doc_window_count <= target_val_windows * 1.1:  
            val_doc_indices.add(doc_idx)
            val_windows_count += doc_window_count
            docs_used.append(doc_idx)
            #print(f"Добавлен документ {doc_idx}: +{doc_window_count} окон (всего: {val_windows_count}/{target_val_windows})")
        
        if val_windows_count >= target_val_windows:
            break
    
    # Остальные документы - в train
    train_doc_indices = set(unique_docs) - val_doc_indices
    
    print(f"\nИтоговое разделение:")
    print(f"Train: {len(train_doc_indices)} документов")
    print(f"Validation: {len(val_doc_indices)} документов")
    
    # Собираем окна для train и validation
    train_indices = [i for i, doc_idx in enumerate(doc_indices) if doc_idx in train_doc_indices]
    val_indices = [i for i, doc_idx in enumerate(doc_indices) if doc_idx in val_doc_indices]
    
    print(f"Train окон: {len(train_indices)} ({len(train_indices)/total_windows*100:.1f}%)")
    print(f"Validation окон: {len(val_indices)} ({len(val_indices)/total_windows*100:.1f}%)")
    
    # Проверка на утечку
    train_docs_in_val = set([doc_indices[i] for i in train_indices]) & set([doc_indices[i] for i in val_indices])
    print(f"\nПроверка на утечку: {'Утечка' if train_docs_in_val else 'Всё хорошо'}")
    
    # 5. Создание датасетов
    train_encodings = {
        'input_ids': [all_windows[i] for i in train_indices],
        'attention_mask': [[1] * len(all_windows[i]) for i in train_indices],
        'labels': [all_labels[i] for i in train_indices]
    }
    
    val_encodings = {
        'input_ids': [all_windows[i] for i in val_indices],
        'attention_mask': [[1] * len(all_windows[i]) for i in val_indices],
        'labels': [all_labels[i] for i in val_indices]
    }
    
    train_dataset = Dataset.from_dict(train_encodings)
    val_dataset = Dataset.from_dict(val_encodings)
    
    dataset = DatasetDict({
        'train': train_dataset,
        'validation': val_dataset
    })
    
    return dataset, label2id, id2label, label_list

In [12]:
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from transformers import AutoModelForTokenClassification, AutoTokenizer, DataCollatorForTokenClassification
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from torch.utils.data import DataLoader

# Конфигурация
DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
WINDOW_SIZE = 510
STRIDE = 64
BASE_PATH = Path("/Users/artemzmailov/Desktop/NER_IT_Resumes_Project")

print(f" Устройство: {DEVICE}")

def evaluate_validation_set(model_path, dataset_path, tokenizer, group_name, output_directory = ''):
    """
    Оценивает модель на validation датасете (20% документов)
    """
    print(f"\nОценка модели {group_name}...")
    print(f"Модель: {model_path}")
    print(f"Датасет: {dataset_path}")
    
    # Загружаем модель
    model = AutoModelForTokenClassification.from_pretrained(str(model_path))
    model = model.to(DEVICE)
    model.eval()
    
    # Подготавливаем датасет 
    dataset, label2id, id2label, label_list = prepare_dataset_with_sliding_windows(
        str(dataset_path), tokenizer, 
        window_size=WINDOW_SIZE, 
        stride=STRIDE, 
        val_fraction=0.2
    )
    
    val_dataset = dataset['validation']
    print(f"Размер validation датасета: {len(val_dataset)} окон")
    
    # DataLoader
    data_collator = DataCollatorForTokenClassification(tokenizer)
    val_dataloader = DataLoader(
        val_dataset, 
        batch_size=16, 
        collate_fn=data_collator,
        shuffle=False
    )
    
    # Собираем предсказания
    all_predictions = []
    all_labels = []
    
    for batch in val_dataloader:
        batch = {k: v.to(DEVICE) for k, v in batch.items()}
        
        with torch.no_grad():
            outputs = model(**batch)
        
        predictions = torch.argmax(outputs.logits, dim=-1)
        mask = batch['labels'] != -100
        
        all_predictions.extend(predictions[mask].cpu().numpy())
        all_labels.extend(batch['labels'][mask].cpu().numpy())
    
    all_predictions = np.array(all_predictions)
    all_labels = np.array(all_labels)
    
    # Метрики по классам
    labels_list = list(id2label.keys())
    precision, recall, f1, support = precision_recall_fscore_support(
        all_labels, all_predictions, labels=labels_list, average=None
    )
    
    # DataFrame
    metrics_df = pd.DataFrame({
        'Сущность': [id2label[i] for i in labels_list],
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'Support': support
    })
    
    # Сортируем по F1
    metrics_df = metrics_df.sort_values('F1-Score', ascending=False)
    
    print("\nМЕТРИКИ ПО КЛАССАМ:")
    print(metrics_df.round(4).to_string(index=False))
    
    # Сохраняем
    output_file = output_directory + f"validation_metrics_{group_name}.csv"
    metrics_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"\nРезультаты сохранены в {output_file}")
    
    return metrics_df

# Запуск

if __name__ == "__main__":
    tokenizer = AutoTokenizer.from_pretrained("Gherman/bert-base-NER-Russian")
    
    models_config = [
        {
            'group': 'group1',
            'model_path': BASE_PATH / 'models' / 'model_1_final',
            'dataset_path': BASE_PATH / 'datasets' / 'group1_dataset.jsonl'
        },
        {
            'group': 'group2',
            'model_path': BASE_PATH / 'models' / 'model_2_final',
            'dataset_path': BASE_PATH / 'datasets' / 'group2_dataset.jsonl'
        },
        {
            'group': 'group3',
            'model_path': BASE_PATH / 'models' / 'model_3_final',
            'dataset_path': BASE_PATH / 'datasets' / 'group3_dataset.jsonl'
        }
    ]
    
    all_results = {}
    for config in models_config:
        try:
            metrics = evaluate_validation_set(
                model_path=config['model_path'],
                dataset_path=config['dataset_path'],
                tokenizer=tokenizer,
                group_name=config['group']
            )
            all_results[config['group']] = metrics
        except Exception as e:
            print(f"Ошибка при обработке {config['group']}: {e}")
    
    # Сводная таблица
    print("\n" + "="*80)
    print("Сводная таблица по всем моделям")
    print("="*80)
    
    summary = []
    for group, df in all_results.items():
        entity_metrics = df[df['Сущность'] != 'O']
        summary.append({
            'Модель': group,
            'Macro F1': f"{entity_metrics['F1-Score'].mean():.4f}",
            'Weighted F1': f"{(df['F1-Score'] * df['Support']).sum() / df['Support'].sum():.4f}",
            'Всего классов': len(df)
        })
    
    summary_df = pd.DataFrame(summary)
    print(summary_df.to_string(index=False))

 Устройство: mps

Оценка модели group1...
Модель: /Users/artemzmailov/Desktop/NER_IT_Resumes_Project/models/model_1_final
Датасет: /Users/artemzmailov/Desktop/NER_IT_Resumes_Project/datasets/group1_dataset.jsonl


Token indices sequence length is longer than the specified maximum sequence length for this model (621 > 512). Running this sequence through the model will result in indexing errors


Подготовка данных с перекрывающимися окнами...
Загружено 307 предложений из JSONL файла
Загружено 307 исходных документов
Найдено 13 меток: ['O', 'B-DEGREE', 'B-LINKS', 'B-LOCATION', 'B-METRICS', 'B-POSITIONS', 'B-TIME', 'I-DEGREE', 'I-LINKS', 'I-LOCATION', 'I-METRICS', 'I-POSITIONS', 'I-TIME']

Статистика по окнам:
Всего окон: 9785
Среднее число окон на документ: 31.9
Мин/макс окон на документ: 1 / 366

Целевой размер валидации: 1957 окон (20%)

Итоговое разделение:
Train: 229 документов
Validation: 78 документов
Train окон: 7825 (80.0%)
Validation окон: 1960 (20.0%)

Проверка на утечку: Всё хорошо
Размер validation датасета: 1960 окон

МЕТРИКИ ПО КЛАССАМ:
   Сущность  Precision  Recall  F1-Score  Support
          O     0.9932  0.9936    0.9934   411595
    I-LINKS     0.9880  0.9938    0.9909    24818
     I-TIME     0.9848  0.9907    0.9877    15313
    B-LINKS     0.9718  0.9769    0.9744     1906
     B-TIME     0.9627  0.9795    0.9710     4242
   B-DEGREE     0.9496  0.9720    